<img src="https://github.com/christopherhuntley/DATA6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **DATA 6510**
# **Lesson 8: Alternative Database Models** 
_NoSQL and SQL Extensions_

## **Learning Objectives**
### **Theory / Be able to explain ...**

- NoSQL technologies like Key-Value Stores, Document Stores, and Graph Databases
- Recent SQL extensions geared towards analytical applications 

### **Skills / Know how to ...**
- Use window functions to work with timestamped data
--------

## **BIG PICTURE: Design Tradeoffs**

Design is about making decisions. If we make the right decisions, then the right systems get built and everybody is blissfully happy. We might not get the credit but people are happy nonetheless. If we make the wrong decisions then everybody is upset *at us*.

So what is the right way to design a system? Well, if that were answerable in a paragraph, then it wouldn't be design. We have to consider what is being asked of the system, what solutions are available, and what we can afford. In other words, it comes down to tradeoffs and priorities. 

We will now take a look at a few eternal data design priorities, in what should be increasing importance for most applications. However, your mileage may vary depending on the needs at the time. 

### **Minimizing Space**


In the old days before big data, storage was often the most expensive part of a computer system. Programmers would do just about any amount of programming to avoid buying new storage hardware. They would literally count characters to minimize the number of bytes a given file required on disk. 


To this end, they came up with some tricks that often shaved off kilobytes without having to resort to file compression. A few examples:
- Repeating fields, where each line of file only recorded what was different from the line above.
- Cryptic codes in place of long strings of text. Often they were hardwired into the programs, working like magic incantations when used by people in the know. 
- Overloading fields so that multiple facts could be stored in one field.

You can see this same kind of thinking today in the messages passed between the front end app and a server. However, network bandwidth is becoming so plentiful that even this last bastion of space efficiency is just not important to worry about. Space is cheap and getting cheaper. 

### **Maximizing Calculation Speed**

Along the same lines as with space, raw speed has historically been prioritized over correctness. Long ago it was because computers were so slow. These days it is because we ask so much more of our computer systems. If we can shave 5% of the computing time off a given operation that will be performed billions of times, then it is well worth it to do so.

Relevant techniques for raw speed include:
- Precomputing whatever can be done in advance, even when it swells storage with redundant data.
- Approximating results whenever 100% fidelity is not strictly necessary.
- Locating data closer to each user, even when it means some data will be out of sync with others

Of course, computers are getting faster and faster. However, expect this trend to continue as demands for raw speed will likely increase faster than we can build bigger and faster hardware.

### **Maximizing Coherency**

Coherency is the ability to make sense of the data. Do all the facts fit together to tell coherent stories? Is each fact expressed in the best possible way? 

Generally, data coherency has been the domain of data modelers, who are more concerned with the stories than the data itself:
- What are the entities being tracked?
- What data is collected about each one? 
- How do the entities relate to each other?

These sorts of questions never get old. They are focused on the same things as the app developer and the data scientists. 

### **Minimizing Risk of Data Corruption**

Data integrity is an essential quality that any analyst should cherish. It is literally seeking to put the truth (and only the truth) into our databases. It is getting harder and harder to achieve, however. 

Big data is ugly data. It often comes in corrupted, forcing the database system to clean it up before it can be stored. If the system is going to do that then it needs to have a goal, a definition of what *correct* and *clean* are. If data can't be fixed then the system should reject it rather than accept a lie as the truth. 

It is this last design priority that is at the heart of table design and normalization. If we design our tables so that they follow a few (not-so-easy) rules, then we can avoid the vast majority of data corruption errors or, as we will call them, **data anomalies**.  


The rest of this lesson -- alas, our last -- will explore a few technologies that address these sorts of tradeoffs with alternatives or extensions to the grand old Relational Data Model. 

---
## **NoSQL Databases**

The term "NoSQL" was first coined in the late 1990s and gained popularity among application programmers about a decade later. It refers to databases that do not rely on the relational model. That does not mean that SQL (or some close approximation) isn't used, but rather, that NoSQL systems extend beyond the traditional relational model. 

For this reason some interpret NoSQL as "Not only SQL" rather than exclusively no use of SQL at all. In fact, each of the NoSQL technologies surveyed here *could* in fact be implemented in SQL, and we will try to use relational models to explain how each relates to SQL and how it extends beyond it. 


Why would we even need to go beyond SQL? 
- **Developer Experience (DX):** As we have discussed before, there is a natural impedance mismatch between a declarative language like SQL and a more imperative application development language like Python, Java, or JavaScript. NoSQL technologies remove much of the discomfort some programmers feel when using SQL.
- **Performance:** Selectively relaxing the rules of the relational model can sometimes bring speed and cost benefits that outweigh the integrity protections of the relational model. 

For each of the models below we will discuss how it differs from the standard relational model, potential DX or performance benefits, and the most common usage scenarios.

### **Key-Value Stores**

Key-Value (KV) stores are most useful when a table is very sparse, like this section of the NBA PlayLog data set. 

![Sparse Table](./img/L11_Sparse_Table.png)

If most values in a table are blank, then why waste time and effort recording them in rows and columns. Instead, just store the data you actually have, tagged to suit however you may need to retrieve it.

In relational terms, a KV store is just a single two column table   
- kv_store(**key**, value)

where
- **key** is a unique index
- value is a datum to be stored

> **Heads Up:** We discussed relational shorthand notation in Lesson 5. 

You may be wondering how we can replace a two-dimensional table with rows and columns with a one dimensional KV store. We do it exactly like we would with composite indexes in the relational model, with the row and column encoded in the key. It's all in the patterns we use when constructing the keys.

For example, the following could be used to record the `assist`,`away`, etc. columns as key-value pairs:

| **Key** | **Value**|
| --- | ---|
| 2:away | Derrick Favors |
| 2:home | Marc Gasol |
| 18:opponent | Kyle Lowry |
| 20:num | 1 |
| 21:num | 2 |

KV Stores are commonly used for creating data caches for the web. Here, for example, is the data that my web browser is keeping about *this page* (somewhat redacted) while I have it open in Google Colab (another Jupyter notebook client): 

![](./img/L11_Colab_Local_Storage.png)

Similar caching technology is used by web servers to minimize reads from the disk storage. If the same CSS is used on every page then why read it from disk every time? The server doesn't. Instead it serves the CSS from a highly-optimized, in-memory KV cache.

#### **Wide-Column KV Stores**
Wide-Column data stores extend the KV model to allow data storage as multiple columns:
- The keys are just like any other KV store.
- The columns are encoded like a SQL `STRUCT` (or JSON object), with each column having a name and a value. 

The columns for each key may vary, like this:

| **Key** | **Value**|
| --- | ---|
| 2 | {away:Derrick Favors, home: Marc Gasol} |
| 18 | {opponent:Kyle Lowry} |
| 20 | {num:1} |
| 21 | {num:2} |

The effect is to condense the already very space-efficient key-value model by eliminating overlapping keys. In this case we eliminated a row of data storage by sharing the key `2` for the `away` and the `home` columns. 

#### **Summary**

- Pros
  - very fast storage and retrieval (when applicable)
  - compact storage 
  - programmer-friendly
- Cons
  - schema by convention instead of rules
  - potential for schema drift as new key types proliferate
  - no FKs or analogous way to integrate data across keys
- When to use
  - with sparse or highly volatile data where the keys need to be flexible
  - as local data stores in application development
  - when caching data to speed up application performance
- Example Products
  - [Varnish](https://varnish-cache.org/)
  - Nginx
  - Squid
  - Memcached
  - redis
  - AWS DynamoDB

### **Document Stores**

Document stores build on the KV model to construct arbitrarily complex (and data rich) databases of *semi-structured* data. Semi-structured data has a schema (so we can interpret and process it), just like any other data model, but not one we can know about in advance. Specifically, in a document store each document (roughly equivalent to a table row) can have its own schema, structuring the data however it likes. We can then only know a document schema *after* opening the document, whereas for the relational model all table schemas are known *before* doing any DML operation. 

The classic example of a document storage format is that used by Microsoft Word, which acts as a *container* for components (blocks of text, titles, images, tables, etc.). One can insert just about anything inside an MS Word file (even malware, unfortunately). MS Word will then compose and render the components as documents in real time so end users can read and edit the data contained inside.

A more relevant example is JSON, which has become the de facto standard format for transmitting data over the web. Like with an MS Word document, a JSON object or list acts as a container, into which we can insert ... JSON objects and lists. The result is a tree structure, with objects and lists nested inside each other to some unspecified depth.

Here, for example, is what a Jupyter notebook (this one, in fact) looks like when you open it in a text editor:

![Jupyter as JSON](./img/L11_Colab_As_JSON.png)

Yes, the `ipynb` file format is really just highly structured JSON. Here's the same JSON in a nicer, pretty printed format:

![Prettified JSON](./img/L11_Pretty_JSON.png)

A Jupyter notebook is mostly a list of "cells" $-$ look for it in the screenshot $-$ where each cell has a `cell_type`, `metadata`, and `source`. The `source` is whatever we typed into the cell.

#### **Summary**

- Pros
  - little or no impedance mismatch for programmers, especially when using JavaScript
  - very compact, especially for semi-structured data
- Cons
  - same as KV stores; complex queries are especially difficult
  - schema on read complicates app design and development; potentially buggy
- When to use
  - for local storage or web transmission of data
  - for complex hierarchically-structured data, where documents are composed of nested components
  - When storing "objects" in relational databases
- Example Products
  - CouchDB
  - mongoDB
  - AWS DynamoDB
  - Google Cloud Firestore

### **Graph Databases**

A graph database organizes data into three kinds of structures:
- A set of **nodes** that represent entities within the domain
- A set of **edges** (or arcs) that connect nodes to represent relationships
- **Properties** that represent the attributes of each entity or relationship

| ![Neo4J Screenshot](./img/L11_Neo4J_Screenshot_Annotated.png) |
|:---:|
| Original Image Source: [*Graph Databases, Linked Data, RDF, and the Semantic Web Wasteland*]( https://medium.com/@eisenzopf/graph-databases-linked-data-rdf-and-the-semantic-web-wasteland-69e9f4347a5b) |

On a superficial level, graph databases are a lot like relational databases, with nodes = entities, properties = attributes, and edges = relationships.  A graph database, however, is much more flexible in how it defines these things: 
- Since we are not using relations, there is no need to separate the entities by type. If we like, **we can treat each node as a unique type, with its own distinct properties (kept as Key-Value pairs).** 
- Whereas a relational database treats relationships as being between entity types, graph databases define them on specific node pairs. **Any pair of nodes can be connected by an edge, regardless of type.** 
- While each node has a unique identifier, there is no concept of type-based foreign keys. Instead, **each edge is treated like an entity with a unique identifier.** The edge then has two implicit properties to reference the nodes it connects. 

With all that said, a graph database *can* be mapped into a very specific relational model where:
- There are three types of entities: nodes, edges, and properties.
- Edge entities are directional, with each having a start node and an end node.
- Properties are associated with nodes or edges as needed.

So if the graph database is really just a special case of the relational model, then what's the point? For some applications a graph database can be much, much faster than a relational database. For example, consider the database behind the [Six Degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game, where each working actor has a *Bacon Number*, BN, that represents the number of edges needed to connect them to Kevin Bacon. Kevin Bacon's BN is 0. Anyone who has appeared in a movie with him has a BN of 1. Anyone who has appeared with anyone whose BN=1 has a BN of 2, etc. The game is to guess an obscure actor's BN, with super bonus points for finding a movie actor *anywhere in the world* without a BN. 

Let all the movie credits in the world be stored in a single table called `credits`, with one row per actor and movie title. To keep things as simple as possible we will assume that 
- Each actor and each movie title has a unique identifier (`actor_id` and `movie_id`).
- We already know the `actor_id` identifiers for Kevin Bacon (`kevin_bacon_id`) and the other actor in question (`other_actor_id`).

Then to find all people with BN=1 we would join the credits table with itself one time:

```sql
-- Check for BN=1
SELECT distinct c1.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 USING (movie_id)
WHERE c0.actor_id = kevin_bacon_id AND c1.actor_id = other_actor_id
```

(If we want to look for a particular actor, we can just include them as `c1.actor_id` in the `WHERE` clause. If there are no results then the actor's BN is greater than 1.) 

To find all the movies that BN=1 actors appeared in would we add in another join:
```sql
SELECT distinct c2.movie_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id)
    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
WHERE c0.actor_id = kevin_bacon_id
```

We can then repeat the process with another join to get the BN=2 actors. To get the BN=3 actors we would add on two more joins to the chain. If we repeat the process out to BN=6 (the theoretical maximum for a working movie actor) then we would have 11 joins in the chain.

```sql
SELECT distinct c11.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id) # BN=1

    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
    JOIN credits AS c3 ON (c2.movie_id = c3.movie_id) # BN=2

    JOIN credits AS c4 ON (c3.actor_id = c4.actor_id)
    JOIN credits AS c5 ON (c4.movie_id = c5.movie_id) # BN=3

    JOIN credits AS c6 ON (c5.actor_id = c6.actor_id)
    JOIN credits AS c7 ON (c6.movie_id = c7.movie_id) # BN=4

    JOIN credits AS c8 ON (c7.actor_id = c8.actor_id)
    JOIN credits AS c9 ON (c8.movie_id = c9.movie_id) # BN=5

    JOIN credits AS c10 ON (c9.actor_id = c10.actor_id)
    JOIN credits AS c11 ON (c10.movie_id = c11.movie_id) # BN=6
WHERE c0.actor_id = kevin_bacon_id AND c11.actor_id = other_actor_id
```

Running up to six queries like this seems ugly but still plenty doable, until you realize that the `credits` table might have 50 million rows. **Even with supercomputer hardware the query could take virtually forever.** With a graph database and a well-tuned search algorithm, however, we can find the specific sequence of edges connecting any actor to any other in less than a second. We can make it *even faster* if we assume that one of the actors is Kevin Bacon. 

The secret is in the specificity of the graph database model. We don't need to explore all possible actor-to-movie-to-actor connections. Instead we only need to follow the connections along the shortest path, which can be found fairly quickly via dynamic programming. If we want it even faster we can try A* search with an "explore big cast movies first" strategy.

The achilles heel of the graph database model is bulk computation. If we need to do the same repetitive operation millions of times, perhaps concurrently, then a relational or perhaps a document database is likely a better fit, depending on the operation in question. Graph databases are designed for representation and searching, and they do those things very well, but not so much for statistical modeling or similar table-oriented applications. 

#### **Summary**

- Pros
  - flexible data model, with nodes, entities, and properties as first-class entity types
  - scales well for associative datasets with lots of relationships
- Cons
  - same as KV stores; complex queries are especially difficult 
  - while searchable, lacks most of the computational power of the relational model
  - treating each edges and property as a separate entity may not be space efficient
- When to use
  - whenever the data is naturally represented as nodes, edges, and properties
  - for network-focused apps like customer relationship management, road navigation, certain search engine operations, and file systems
  - for semantic maps (e.g., in text analytics) where the relationships represent interactions, forces, hierarchies, etc.
- Example Products
  - Neo4j
  - RedisGraph
  - AWS Neptune


---
## **Note about Collection Types**



In lesson 5 we learned that entities should be atomic, indivisible through normalization. Among other things this means that attributes should be singular, represeting just one fact about a given entity. Plural attributes, meanwhile, should be normalized out into one-to-many relationships to new entity types. 

But what if we just didn't bother to do that? What if we want to go semi-structured, without a fixed data structure because the data itself varies too much? You, know, like in Python.

The `ARRAY` and `STRUCT` collection types were added to the SQL standards in a wave of enthusiasm for object-relational features in SQL:1999 but then (like window functions) largely ignored by most database vendors until 2018. Why ignore the standard? Because collection types permit us to skirt around a fundamental law of normalization: thou shalt not use repeated fields. 

An `ARRAY` is equivalent to a Python list. The iteral syntax is even the same: for the NBA example a lineup could be represented by 

```sql
['Kyle Lowry','Fred VanVleet','OG Anunoby','Pascal Siakam','Marc Gasol']
```

A `STRUCT` is then the equivalent to a Python dictionary. Once again, here is the NBA example with positions as keys:

```sql
{'p1': 'Kyle Lowry', 'p2': 'Fred VanVleet', 'p3': 'OG Anunoby', 'p4': 'Pascal Siakam', 'p5': 'Marc Gasol'}
```

We can nest collections inside of one another to create complex data structures. In fact, many database vendors include utilities to translate directly to JSON. 

It is generally not a good idea to use these sorts of collection types in a transactional database, where vigilance is needed to keep the data consistent and anomaly free. However, for dimensional data warehouses, where the **dimension tables** are often denormalized anyway, they make **perfect sense**: no extra joins, no redundant columns, etc., just the relevant context in a concise format. 

***Technically*, however, collection types do not actually violate normalization rules.** A collection is a **container** that can hold structured data. The container itself is singular, though the contents might be plural. It's like how you might keep a plastic *container* of cut celery sticks instead of storing them in a pile on the shelf. The container is *one* thing (with the sticks nested inside), but you have to open the container (unpack it) to get your snack. That's exactly how SQL `ARRAY` and `STRUCT` work. We have to use functions and special syntax to pack and unpack data. Otherwise, SQL just treats it like an image or other binary data. Of course, it *is* still a problem that we don't know what's inside the container before we unpack it but we can still say the table itself is in first normal form. 

---
## **Congratulations! You've made it to the end of the course.**

Believe it or not, we're all done. There is **a lot more** to learn about SQL (e.g., `CREATE`, `INSERT`, `UPDATE`, `DELETE`, `DROP`) but you will need to learn it elsewhere. 

I hope you have enjoyed learning just enough SQL. 